In [ ]:
# KELOMPOK 14
# Nama:
# 1. David Hasudungan Lumbangaol (124400004)
# 2. Tino Sumandi Siregar        (124400036)
# 3. Amjad Rozan                 (124400016)


# JUDUL PROJEK :  SISTEM KASIR SEDERHANA BERBASIS PYTHON


# DESKRIPSI PROJEK :
# Program ini merupakan sistem kasir sederhana dengan Python 
# yang dijalankan di Jupyter Notebook menggunakan ipywidgets.
# Data produk diambil dari file CSV lokal (produk.csv).
# Jika file tidak ditemukan, program akan berhenti.

# Program ini memiliki fitur:
# - Menambah produk ke keranjang
# - Melihat isi keranjang
# - Menghitung total belanja + diskon otomatis
# - Menyimpan transaksi ke file JSON
# - Mengurangi stok produk setelah dibeli
# - Percabangan, perulangan, fungsi, lambda, dan exception handling

In [7]:
# IMPORT LIBRARY
import csv
import json
import os
import ipywidgets as widgets
from IPython.display import display, clear_output


In [15]:
# LOAD DATA PRODUK 
produk_list = []
file_path = "produk.csv"

print("Memuat data produk...")

try:
    with open(file_path, mode='r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            try:
                row['harga'] = float(row['harga'])
                row['stok'] = int(row['stok'])
                produk_list.append(row)
            except:
                print("Error: Data CSV tidak valid.")
                raise
    if not produk_list:
        raise ValueError("CSV kosong.")
except FileNotFoundError:
    raise FileNotFoundError("ERROR: File 'produk.csv' tidak ditemukan!")
except Exception as e:
    raise e

print("Data produk berhasil dimuat!\n")
# FUNGSI UTAMA
keranjang = []

def hitung_total(keranjang):
    return sum(item['harga'] * item['jumlah'] for item in keranjang)

# Lambda diskon
diskon = lambda total: total * 0.9 if total > 50000 else total
# OUTPUT AREA
output_area = widgets.Output()
def tampilkan_menu():
    with output_area:
        clear_output()
        print("=== MENU KASIR ===")
        print("1. Tambah produk")
        print("2. Lihat keranjang")
        print("3. Hitung total & bayar")
        print("4. Reset keranjang\n")

        if keranjang:
            print("ISI KERANJANG:")
            for item in keranjang:
                print(f"- {item['nama']} ({item['jumlah']} x {item['harga']})")
        else:
            print("Keranjang masih kosong.")
# EVENT: TAMBAH PRODUK
def tambah_produk(b):
    with output_area:
        nama = input_nama.value
        try:
            jumlah = int(input_jumlah.value)
        except ValueError:
            print("\nJumlah harus angka!")
            return
        
        # Cek produk di stok
        ditemukan = next((p for p in produk_list if p['nama'].lower() == nama.lower()), None)
        
        if not ditemukan:
            print("Produk tidak ditemukan!")
            return
        
        if jumlah > ditemukan['stok']:
            print("Stok tidak cukup!")
            return
        
        # Tambahkan ke keranjang
        keranjang.append({
            'nama': ditemukan['nama'],
            'harga': ditemukan['harga'],
            'jumlah': jumlah
        })

        # Kurangi stok
        ditemukan['stok'] -= jumlah

        print(f"{nama} sebanyak {jumlah} ditambahkan ke keranjang!")

        input_nama.value = ""
        input_jumlah.value = ""

        tampilkan_menu()

# EVENT: MENU BUTTON
def menu_click(b):
    pilihan = b.description

    with output_area:
        clear_output()

        if pilihan == "Tambah Produk":
            print("Masukkan Nama Produk & Jumlah:")
            display(input_nama, input_jumlah, btn_tambah)

        elif pilihan == "Lihat Keranjang":
            tampilkan_menu()

        elif pilihan == "Hitung Total":
            if not keranjang:
                print("Keranjang kosong!")
            else:
                total = hitung_total(keranjang)
                total_d = diskon(total)
                print(f"Total sebelum diskon : {total}")
                print(f"Total setelah diskon  : {total_d}\n")

                # Simpan transaksi
                with open("transaksi.json", "w") as f:
                    json.dump({"keranjang": keranjang, "total": total_d}, f)

                print("Transaksi disimpan ke 'transaksi.json'.")
                
                keranjang.clear()

            tampilkan_menu()

        elif pilihan == "Reset":
            keranjang.clear()
            print("Keranjang dikosongkan!")
            tampilkan_menu()
# WIDGET
btn1 = widgets.Button(description="Tambah Produk")
btn2 = widgets.Button(description="Lihat Keranjang")
btn3 = widgets.Button(description="Hitung Total")
btn4 = widgets.Button(description="Reset")

menu_box = widgets.HBox([btn1, btn2, btn3, btn4])

input_nama = widgets.Text(description="Nama:")
input_jumlah = widgets.Text(description="Jumlah:")
btn_tambah = widgets.Button(description="Tambah")

btn1.on_click(menu_click)
btn2.on_click(menu_click)
btn3.on_click(menu_click)
btn4.on_click(menu_click)
btn_tambah.on_click(tambah_produk)
#TAMPILAN NENU AWAL
display(menu_box)
display(output_area)
tampilkan_menu()
# TAMPILKAN TRANSAKSI TERAKHIR
print("\nMengecek transaksi terakhir...")
try:
    with open("transaksi.json", "r") as f:
        print("Transaksi terakhir:")
        print(json.load(f))
except:
    print("Belum ada transaksi atau file rusak.")
# VISUALISASI STOK
print("\n=== VISUALISASI STOK PRODUK ===")
print("Nama Produk | Stok | Grafik")
print("-------------------------------------")
for p in produk_list:
    bar = '*' * (p['stok'] // 2)
    print(f"{p['nama']:<12} | {p['stok']:<4} | {bar}")



Memuat data produk...
Data produk berhasil dimuat!



Output()


Mengecek transaksi terakhir...
Transaksi terakhir:
{'keranjang': [{'nama': 'Jeruk', 'harga': 7000.0, 'jumlah': 15}], 'total': 94500.0}

=== VISUALISASI STOK PRODUK ===
Nama Produk | Stok | Grafik
-------------------------------------
Apel         | 10   | *****
Jeruk        | 15   | *******
Pisang       | 20   | **********
Mangga       | 5    | **


In [16]:
# KESIMPULAN
print("\nKESIMPULAN:")
print("Program sistem kasir sederhana berjalan dengan baik.")
print("Fitur: tambah produk, lihat keranjang, hitung total + diskon, simpan transaksi JSON,")
print("stok berkurang otomatis, visualisasi stok, percabangan, fungsi, loop, dan exception handling.")


KESIMPULAN:
Program sistem kasir sederhana berjalan dengan baik.
Fitur: tambah produk, lihat keranjang, hitung total + diskon, simpan transaksi JSON,
stok berkurang otomatis, visualisasi stok, percabangan, fungsi, loop, dan exception handling.
